In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8561080726515263681
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 772538368
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6905870099506070564
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


2022-02-23 16:02:57.272107: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-23 16:02:58.131305: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:0 with 736 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1


In [3]:
bar = tqdm(total=4)
bar.set_description("Raspakek data")

inputLines = np.load('traindata/inputLines.npy')/256
bar.update(1)

service = np.load('traindata/service.npy')
bar.update(1)

binaryData = np.load('traindata/binaryData.npy')
bar.update(1)

#blockL0 = np.load('traindata/blockL0.npy')
#bar.update(1)
#
#blockR0 = np.load('traindata/blockR0.npy')
#bar.update(1)
#
#blockL1 = np.load('traindata/blockL1.npy')
#bar.update(1)
#
#blockR1 = np.load('traindata/blockR1.npy')
#bar.update(1)
#
#blockL2 = np.load('traindata/blockL2.npy')
#bar.update(1)
#
#blockR2 = np.load('traindata/blockR2.npy')
#bar.update(1)

#target_data = {'service': service, 'blockL0': blockL0, 'blockR0':blockR0, 'blockL1':blockL1, 'blockR1':blockR1, 'blockL2':blockL2, 'blockR2':blockR2}
target_data = {'service': service, 'binaryData':binaryData}
bar.update(1)
bar.close()

Raspakek data: 100%|██████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.20s/it]


In [4]:
#remove all data after 100000 pos
if False:
    remAftr = 100000
    inputLines = inputLines[:remAftr]
    service = service[:remAftr]
    binaryData = binaryData[:remAftr]
    #blockL0 = blockL0[:remAftr]
    #blockR0 = blockR0[:remAftr]
    #blockL1 = blockL1[:remAftr]
    #blockR1 = blockR1[:remAftr]
    #blockL2 = blockL2[:remAftr]
    #blockR2 = blockR2[:remAftr]
    target_data = {'service': service, 'binaryData':binaryData}

In [5]:
inputLines.shape

(935123, 180)

In [8]:
binaryData[5151]

array([ True, False,  True, False, False,  True, False, False,  True,
       False,  True, False, False, False, False,  True,  True, False,
        True, False,  True, False, False,  True, False, False,  True,
       False,  True, False,  True, False, False, False,  True, False,
        True, False,  True,  True,  True,  True,  True, False,  True,
       False, False, False,  True, False,  True,  True, False,  True,
       False, False, False, False,  True, False,  True,  True, False,
        True, False, False,  True, False, False,  True, False,  True,
       False, False,  True, False, False, False, False, False, False,
        True,  True, False,  True, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True, False,
       False,  True, False, False, False, False,  True, False, False,
       False,  True, False,  True, False,  True,  True, False, False,
       False,  True,

In [9]:
service[5151]

array([ True, False])

In [5]:
model = keras.models.load_model("model")

2022-02-23 16:03:13.888666: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 736 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1


In [12]:
input = keras.Input(180, )
x = layers.Dense(137, activation='linear', use_bias=True, bias_initializer='zeros')(input)
#x = layers.Dropout(0.1)(x)
x = layers.Dense(137, activation='linear', use_bias=True, bias_initializer='zeros')(x)

service = layers.Dense(2 , activation='sigmoid', name='service')(x)
binaryData = layers.Dense(137 , activation='sigmoid', name='binaryData')(x)
#blockL0 = layers.Dense(16, activation='sigmoid', use_bias=True, bias_initializer='zeros', name='blockL0')(x)
#blockR0 = layers.Dense(16, activation='sigmoid', use_bias=True, bias_initializer='zeros', name='blockR0')(x)
#blockL1 = layers.Dense(16, activation='sigmoid', use_bias=True, bias_initializer='zeros', name='blockL1')(x)
#blockR1 = layers.Dense(16, activation='sigmoid', use_bias=True, bias_initializer='zeros', name='blockR1')(x)
#blockL2 = layers.Dense(16, activation='sigmoid', use_bias=True, bias_initializer='zeros', name='blockL2')(x)
#blockR2 = layers.Dense(16, activation='sigmoid', use_bias=True, bias_initializer='zeros', name='blockR2')(x)


#model = keras.Model(input, [service, blockL0, blockR0, blockL1, blockR1, blockL2, blockR2])
model = keras.Model(input, [service, binaryData])


model.summary()

tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
    layer_range=None,
    show_layer_activations=True,
)

#visualizer(model, format='png', view=True)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 180)]        0           []                               
                                                                                                  
 dense_2 (Dense)                (None, 137)          24797       ['input_2[0][0]']                
                                                                                                  
 dense_3 (Dense)                (None, 137)          18906       ['dense_2[0][0]']                
                                                                                                  
 service (Dense)                (None, 2)            276         ['dense_3[0][0]']                
                                                                                            

In [13]:

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['binary_accuracy'])


In [7]:
model.fit(inputLines, target_data, epochs=1-0x7fff)

2022-02-23 16:03:34.321553: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 673288560 exceeds 10% of free system memory.
2022-02-23 16:03:34.999109: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 128111851 exceeds 10% of free system memory.
2022-02-23 16:03:45.107026: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 122.18MiB (rounded to 128111872)requested by op SameWorkerRecvDone
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-02-23 16:03:45.107071: I tensorflow/core/common_runtime/bfc_allocator.cc:1027] BFCAllocator dump for GPU_0_bfc
2022-02-23 16:03:45.107084: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (256): 	Total Chunks: 31, Chunks in use: 31. 7.8KiB allocated for chunks. 7.8KiB in use in bin. 136

ResourceExhaustedError: {{function_node __wrapped____EagerConst_device_/job:localhost/replica:0/task:0/device:GPU:0}} SameWorkerRecvDone unable to allocate output tensor. Key: /job:localhost/replica:0/task:0/device:CPU:0;5ded638e35511b02;/job:localhost/replica:0/task:0/device:GPU:0;edge_2_input;0:0

In [10]:
model.save("model")

INFO:tensorflow:Assets written to: model\assets


In [13]:
target_data['binaryData'][50]

array([ True, False,  True, False, False, False, False,  True, False,
       False,  True,  True, False,  True,  True,  True,  True, False,
       False, False,  True, False, False, False, False, False, False,
        True,  True,  True, False,  True, False, False, False,  True,
       False,  True,  True, False, False,  True,  True,  True, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True, False,  True, False, False,  True,  True, False,
       False, False,

In [28]:
print(model.predict(np.array([inputLines[50]])))

[array([[9.994783e-01, 9.570409e-04]], dtype=float32), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0.,
        1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)]
